pre install

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth! obviously
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

create array of pairs for the example questions to tune on

In [2]:
from transformers import AutoTokenizer

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/data/Tuning questions.csv')


FILES = [(row['user_prompt'].replace('data/', '').strip(),
          row['system_response'].replace('data/', '').strip())
         for _, row in df.iterrows()
         if isinstance(row['user_prompt'], str) and len(row['user_prompt'].strip()) > 0 and
            isinstance(row['system_response'], str) and len(row['system_response'].strip()) > 0]

print(FILES)
INSTRUCTION = "Please answer the user_prompt about foster care policy and put a link to the relevant source at the end."


[('When does the policy change regarding the CAPS program transition take effect, and to which counties will it apply?', 'The policy change regarding the CAPS program transition took effect on July 17, 2017. Initially, it applied to 13 counties: Bibb, Brooks, Catoosa, Clarke, Colquitt, Cook, Douglas, Echols, Gilmer, Gordon, Lowndes, Murray, and Whitfield.\n\nPlease see the following link for more information: https://fostercaresystems.wustl.edu/foster/childcare-and-parent-services-caps-program-transition-department-early-care-and-learning'), ('What are the responsibilities and standards for caregivers in the Division of Family and Children Services (DFCS) when it comes to making decisions about age and developmentally appropriate activities for children in foster care?', "Caregivers in the Division of Family and Children Services (DFCS) are responsible for ensuring that children in foster care have ongoing opportunities to engage in age and developmentally appropriate activities, inclu

In [3]:
!pip install torch
from unsloth import FastLanguageModel
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch

if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please ensure you have a CUDA-enabled GPU and the correct drivers installed.")
#debugging


max_seq_length = 8096 # RoPE Scaling
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.0: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters

In [4]:
#yes
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.0 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


format for data prep (more formating)

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

<a name="Data"></a>
### Data Prep
Convert to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```


In [6]:
from datasets import Dataset


# Prepare the dataset with your prompt template.
prompt = """Below is a question about foster care policy in Georgia. Please answer in the same style as me with a link to the relevant source at the end.

### user_prompt:
{}

### system_response:
{}

""" + tokenizer.eos_token

# Create the training data examples
finetuning_examples = []
for pair in FILES:
    user_prompt = pair[0].strip()
    system_response = pair[1].strip()
    finetuning_examples.append({
        "text": prompt.format(user_prompt, system_response)
    })

dataset = Dataset.from_list(finetuning_examples)

# Ensure your dataset is in a format that your model training process expects.
# print(f"Number of examples: {len(dataset)}")
# print(f"Example entry: {dataset[0]}")

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [7]:
def convert_to_conversations(example):
    """
    This function takes each 'text' entry and splits it into a structured
    format with 'role' (user/assistant) and 'content'.
    """
    text = example['text']

    # Split the prompt and response using the predefined markers in the prompt template
    user_prompt_start = "### user_prompt:\n"
    system_response_start = "### system_response:\n"

    # Find where the user prompt starts and system response starts
    user_prompt_index = text.find(user_prompt_start) + len(user_prompt_start)
    system_response_index = text.find(system_response_start)

    # Extract user prompt and system response
    user_prompt = text[user_prompt_index:system_response_index].strip()
    system_response = text[system_response_index + len(system_response_start):].strip()

    # Remove <|eot_id|> if present
    system_response = system_response.replace("<|eot_id|>", "").strip()

    # Return the conversation structure
    return {
        "conversations": [
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": system_response}
        ]
    }

# Apply the conversion function to each entry in the dataset
converted_dataset = dataset.map(convert_to_conversations, remove_columns=["text"])

# Check the output structure
print(converted_dataset[:5])

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

{'conversations': [[{'content': 'When does the policy change regarding the CAPS program transition take effect, and to which counties will it apply?', 'role': 'user'}, {'content': 'The policy change regarding the CAPS program transition took effect on July 17, 2017. Initially, it applied to 13 counties: Bibb, Brooks, Catoosa, Clarke, Colquitt, Cook, Douglas, Echols, Gilmer, Gordon, Lowndes, Murray, and Whitfield.\n\nPlease see the following link for more information: https://fostercaresystems.wustl.edu/foster/childcare-and-parent-services-caps-program-transition-department-early-care-and-learning', 'role': 'assistant'}], [{'content': 'What are the responsibilities and standards for caregivers in the Division of Family and Children Services (DFCS) when it comes to making decisions about age and developmentally appropriate activities for children in foster care?', 'role': 'user'}, {'content': "Caregivers in the Division of Family and Children Services (DFCS) are responsible for ensuring 

In [8]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(converted_dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/19 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [9]:
dataset[5]["conversations"]

[{'content': 'What rights and procedures are outlined for foster parents under the Foster Parent Bill of Rights and Grievance Procedure in Georgia, and how can grievances be formally addressed through the Department of Family and Children Services (DFCS)?',
  'role': 'user'},
 {'content': 'Foster parents in Georgia have rights outlined in the Foster Parent Bill of Rights, which include being treated with dignity and respect, not being discriminated against, continuing with their own family values, receiving appropriate training, being informed of relevant information, receiving timely financial reimbursement, and being notified of hearings and reviews involving the child. Grievances can be formally addressed through a multi-step process:\n\n    1. Foster parents file a written complaint with the local DFCS County Director.\n    2. If the complaint is not resolved within 15 business days, a written complaint is submitted to the State DFCS Division Director.\n    3. If the issue remains 

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [10]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat rights and procedures are outlined for foster parents under the Foster Parent Bill of Rights and Grievance Procedure in Georgia, and how can grievances be formally addressed through the Department of Family and Children Services (DFCS)?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nFoster parents in Georgia have rights outlined in the Foster Parent Bill of Rights, which include being treated with dignity and respect, not being discriminated against, continuing with their own family values, receiving appropriate training, being informed of relevant information, receiving timely financial reimbursement, and being notified of hearings and reviews involving the child. Grievances can be formally addressed through a multi-step process:\n\n    1. Foster parents file a written complaint with the

<a name="Train"></a>
### Train the model
Now use Huggingface TRL's `SFTTrainer`, more docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 20, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/19 [00:00<?, ? examples/s]

Use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

We verify masking is actually done:

In [13]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat rights and procedures are outlined for foster parents under the Foster Parent Bill of Rights and Grievance Procedure in Georgia, and how can grievances be formally addressed through the Department of Family and Children Services (DFCS)?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nFoster parents in Georgia have rights outlined in the Foster Parent Bill of Rights, which include being treated with dignity and respect, not being discriminated against, continuing with their own family values, receiving appropriate training, being informed of relevant information, receiving timely financial reimbursement, and being notified of hearings and reviews involving the child. Grievances can be formally addressed through a multi-step process:\n\n    1. Foster parents file a written complaint with the

In [14]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                             \n\nFoster parents in Georgia have rights outlined in the Foster Parent Bill of Rights, which include being treated with dignity and respect, not being discriminated against, continuing with their own family values, receiving appropriate training, being informed of relevant information, receiving timely financial reimbursement, and being notified of hearings and reviews involving the child. Grievances can be formally addressed through a multi-step process:\n\n    1. Foster parents file a written complaint with the local DFCS County Director.\n    2. If the complaint is not resolved within 15 business days, a written complaint is submitted to the State DFCS Division Director.\n    3. If the issue remains unresolved, the complaint can be escalated to the State Mediation Committee for facilitation and resolution.\n\nThis process ensures that foster parents have a formal mechanism to address grievances and seek r

We can see the System and Instruction prompts are successfully masked!

In [15]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.635 GB of memory reserved.


In [16]:
#train!
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 19 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 40
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,2.346400
2,2.175600
3,2.243400
4,2.139100
5,2.204700
6,1.972800
7,1.845000
8,1.670000
9,1.603900
10,1.594600


In [17]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

205.4333 seconds used for training.
3.42 minutes used for training.
Peak reserved memory = 3.361 GB.
Peak reserved memory for training = 0.726 GB.
Peak reserved memory % of max memory = 22.79 %.
Peak reserved memory for training % of max memory = 4.923 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank



Use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [21]:
#v2
import pandas as pd
from unsloth.chat_templates import get_chat_template
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import TextStreamer


# Step 1: Load the CSV file
file_path = '/content/data/Clean Georgia Data.csv'
df = pd.read_csv(file_path, encoding='MacRoman')

# Step 2: Define a function to search relevant policies based on the user's query
def search_relevant_policies(query, df, top_n=5):
    # Search for relevant policies based on the query in the 'Content' column
    # Convert policies into a TF-IDF matrix
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['Content'])

    # Get the query as a TF-IDF vector
    query_vector = tfidf.transform([query])

    # Calculate cosine similarity between query and policies
    from sklearn.metrics.pairwise import cosine_similarity
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get the top 5 relevant policies can get more
    top_indices = cosine_sim.argsort()[-top_n:][::-1]
    relevant_policies = df.iloc[top_indices]

    # If no relevant policies are found, fallback to a generic response
    if relevant_policies.empty:
        return "No specific policy found for this query."

    # Combine the top N relevant policies, including all columns, into a formatted string
    formatted_policies = []
    for index, row in relevant_policies.iterrows():
        formatted_policy = f"Title: {row['Title']}\nTerritory: {row['Territory']}\nType: {row['Type']}\nYear: {row['Year']}\nCategory: {row['Category']}\nFrom: {row['From']}\nTo: {row['To']}\nContent: {row['Content']}\nLink: {row['Link to Content']}\n"
        formatted_policies.append(formatted_policy)

    # Join all the formatted policies into a single string
    return "\n\n".join(formatted_policies)

# Step 3: User's query
user_query = "How much are foster parents paid to care for foster children"

# Step 4: Retrieve relevant policies based on the query
relevant_policy_text = search_relevant_policies(user_query, df)
# print(relevant_policy_text)
print(relevant_policy_text)


# Step 5: Combine the relevant policies with the user's query
messages_with_relevant_policies = [
    {"role": "system", "content": relevant_policy_text},  # Add relevant policies as system content
    {"role": "user", "content": user_query},
]

# Step 6: Apply chat template and tokenize
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)
inputs = tokenizer.apply_chat_template(
    messages_with_relevant_policies,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
FastLanguageModel.for_inference(model)
outputs = model.generate(input_ids=inputs, streamer = text_streamer, max_new_tokens=512, use_cache=True, temperature=1.5, min_p=0.1)

# Step 8: Decode the output if not using streamer
# generated_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# print(generated_response[0])


Title: GA :: Child Welfare Policy Manual :: Chapter 10 Foster Care :: Section 10.0 Introduction To Foster Care
Territory: Georgia
Type: Rules
Year: 2023, 2022, 2021, 2020
Category: Expectations for care, Services while in care - Other
From: nan
To: nan
Content: CODES/REFERENCESPublic Law 96-272Adoption and Safe Families Act of 1997 Public Law 105-89DISCUSSIONOverviewFoster care is the temporary out of home care for children who cannot live with their families primarily due to safety threats within their own home. The Division of Family and Children Servicesô (DFCS) Foster Care Program through partnership with the family, caregivers, court and other stakeholders provides support and strengthen families, protect children from further child abuse, and support every child having a permanent family.†GOALSEnsure the safety and well-being of children in care;Enhance caregiver protective capacities so caregivers are able to ensure the safety and well-being of their children;Preserve family rel

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the heart of Paris, stands tall among the city's historic and cultural landmarks. This iron structure, standing at an impressive 324 meters high, offers breathtaking views of the City of Light's iconic landscape. The Eiffel Tower was built for the 1889 World's Fair and has since become a symbol of French engineering and culture.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>